# Notes
##### All Items Have
- id
- level _(can be 0)_
- name
- icon
- quality
- class
- subclass
##### Items can have
- __Equipment Info__
  - Equip Slot
  - __Weapon Stats__
    - dmgmin
    - dmgmax
    - speed
    - dps _(can be meta data)_
  - __Attributes__
    - __Primary Attributes__
      - Strength
      - Agility
      - Intellect
      - Stamina
      - Spirit
      - Armor
    - __Secondary Attributes__
      - Melee/Ranged/Spell/All Critical Strike Rating
      - Melee/Ranged/Spell/All Hit Rating
      - Melee/Ranged/All Attack Power
      - Healing/Damage/All Spell Power 
- Vendor Sell Price

## Links
- [Extentions in Webdriver](https://www.reddit.com/r/learnpython/comments/4zzn69/how_do_i_get_adblockplus_to_work_with_selenium/)
- [requests docs](https://requests.readthedocs.io/en/latest/)
- [tqdm docs](https://tqdm.github.io/)
- [concurrent.futures docs](https://docs.python.org/dev/library/concurrent.futures.html)
- [seaborn docs](https://seaborn.pydata.org/api.html)
- [chrome switches](https://stackoverflow.com/questions/38335671/where-can-i-find-a-list-of-all-available-chromeoption-arguments)
- [mathplotlib docs](https://matplotlib.org/stable/api/index.html)

In [ ]:
# Imports
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
from pathlib import Path
import shutil
from tqdm import tqdm
import time
import requests
import xml.etree.ElementTree as ET
import seaborn as sns
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

In [ ]:
# Settings
root_url = r'https://www.wowhead.com/wotlk'
output_dir = r'output'

min_itemlvl = 0
max_itemlvl = 284

In [ ]:
# Setup
Path(output_dir).mkdir(parents=True, exist_ok=True)

driver_options = webdriver.chrome.options.Options()
driver_options.page_load_strategy = 'normal'
driver_options.add_argument(r'--headless')
driver = webdriver.Chrome(options=driver_options)

# Setup item scraping
items_url = root_url + "/items"
driver.get(items_url)

In [ ]:
# Scrape item quality and build itemlist links
def process_item_quality_elem(elem):
    _id = int(elem.get_attribute("value"))
    _name = elem.text
    _color = elem.value_of_css_property("color")
    _color = _color[_color.index('(') + 1:]
    _color = _color[:_color.index(')')]
    _color_split = _color.split(', ')
    _color = '#' + "{:02x}".format(int(_color_split[0])) + "{:02x}".format(
        int(_color_split[1])) + "{:02x}".format(int(_color_split[2]))
    return {"id": _id, "name": _name, "color": _color}

item_qualities_csv_path = Path(output_dir + "/item_qualities.csv")
item_qualities_csv_exists = item_qualities_csv_path.exists()

if(item_qualities_csv_exists):
    item_qualities = pd.read_csv(item_qualities_csv_path, sep=';')
else:
    item_quality_elems = driver.find_elements(
        By.CSS_SELECTOR, '#filter-facet-quality > option')
    item_qualities = pd.DataFrame.from_records(
        [process_item_quality_elem(e) for e in item_quality_elems],
        index='id'
    )
    item_qualities.to_csv(item_qualities_csv_path, sep=';')

item_qualities_palette = sns.color_palette(item_qualities['color'])

itemlist_urls = []
for quality in item_qualities.itertuples():
    for itemlvl in range(min_itemlvl, max_itemlvl):
        itemlist_url = "{u}/min-level:{l:n}/max-level:{l:n}/quality:{q:n}".format(
            u=items_url,
            l=itemlvl,
            q=quality.Index
        )
        itemlist_urls.append(itemlist_url)

In [ ]:
itemlist_data_rgx1 = r'WH\.Gatherer\.addData\(.*?\)'

In [ ]:
get_listview_data_json_js = r'return JSON.stringify(g_listviews.items.data)'
def process_itemlist2(url):
    driver.get(url)
    try:
        listview_data_json = driver.execute_script(get_listview_data_json_js)
    except:
        return []
    else:
        return json.loads(listview_data_json)

itemlist_datas = []


for ilvl in tqdm(range(min_itemlvl, max_itemlvl), desc="Iterating item lists", leave=False):
    u = root_url + f'/items/min-level:{ilvl}/max-level:{ilvl}'
    data = process_itemlist2(u)
    data_len = len(data)
    if data_len == 0:
        continue
    elif data_len < 1000:
        itemlist_datas += data
    else:
        for qid in item_qualities['id']:
            u += f'/quality:{qid}'
            itemlist_datas += process_itemlist2(u)

            

# for u in tqdm(itemlist_urls, desc="Iterating item lists for item urls", leave=False):
#     itemlist_data = process_itemlist2(u)

#     itemlist_datas.append()

In [ ]:
# Scrape itemslists for item urls
def process_itemlist(url):
    click_next_btn_js = r'next_btn = Array.from(document.querySelectorAll("#tab-items > div.listview-band-top > div.listview-nav > a")).find(x => x.textContent.toLowerCase().startsWith("next")); if(next_btn !== undefined) next_btn.click()'
    get_item_links_js = r'return Array.from(document.querySelectorAll("#tab-items > div.listview-scroller-horizontal > div > table > tbody > tr > td:nth-child(2) > div > a")).map(x => x.href)'
    driver.get(url)
    frames = []
    pre_url = ""
    # TODO check and ensure that less < 1000 items where returned by query
    while driver.current_url != pre_url:
        item_links = driver.execute_script(get_item_links_js)
        frames.append(pd.DataFrame(item_links))
        pre_url = driver.current_url
        driver.execute_script(click_next_btn_js)

    return pd.concat(frames)


item_urls_csv_path = Path(output_dir + "/item_urls.csv")
item_urls_csv_exists = item_urls_csv_path.exists()
if (item_urls_csv_exists):
    item_urls = pd.read_csv(item_urls_csv_path, header=None).rename(columns={0: "url"})
else:
    item_url_frames = []
    for u in tqdm(itemlist_urls, desc="Iterating item lists for item urls", leave=False):
        item_url_frames.append(process_itemlist(u))

    item_urls = pd.concat(item_url_frames).drop_duplicates().rename(columns={0: "url"})
    item_urls.sort_values(by=['url'], inplace=True, ignore_index=True)
    item_urls.to_csv(item_urls_csv_path, sep=';', index=None, header=None)

In [ ]:
# Scrape item XML setup
item_xml_dir = Path(output_dir + "/itemxml")
item_xml_dir.mkdir(parents=True, exist_ok=True)

def get_item_xml_info(item_url):
    idx1 = item_url.index('item=', len(root_url))
    idx2 = item_url.index('/', idx1)
    item_xml_url = item_url[:idx2] + "&xml"
    li = item_xml_url.index(r'item=') + len(r'item=')
    ri = item_xml_url.index(r'&xml')
    item_id_str = item_xml_url[li:ri]
    item_xml_filename = item_id_str + r'.xml'
    item_xml_filepath = Path(str(item_xml_dir) + '/' +
                             item_xml_filename).absolute()
    return {r'url': item_xml_url, r'filepath': item_xml_filepath}

def download_item_xml(item_url, overwrite=False):
    item_xml_info = get_item_xml_info(item_url)
    item_xml_exists = item_xml_info['filepath'].exists()
    if (not item_xml_exists or overwrite):
        rsp = requests.get(item_xml_info['url'])
        rsp.raise_for_status()
        item_xml_info['filepath'].write_bytes(rsp.content)
    return item_xml_info

In [ ]:
# Scrape item XML download
item_xml_paths = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=(os.cpu_count() - 1)) as executor:
    fail_count = -1
    while fail_count != 0:
        fail_count = 0
        futures = [executor.submit(download_item_xml, u, False) for u in item_urls['url']]
        for future in tqdm(iterable=concurrent.futures.as_completed(futures), desc="Downloading Item XML", total=len(futures), leave=False):
            try:
                fres = future.result()
            except:
                fail_count += 1
            else:
                item_xml_paths[str(fres['filepath'])] = 1
item_xml_paths = pd.DataFrame([k for k in item_xml_paths.keys()])[0]

In [ ]:
# shutdown
driver.close()

In [ ]:
# parsing item XML
json_dir = Path(output_dir + "/itemjson")
json_dir.mkdir(parents=True, exist_ok=True)
jsonequip_dir = Path(output_dir + "/jsonequip")
jsonequip_dir.mkdir(parents=True, exist_ok=True)

def parse_item_xml(path, overwrite_json=False):
    root = ET.parse(path).find(".//item")
    class_elem = root.find("class")
    subclass_elem = root.find("subclass")
    quality_elem = root.find("quality")
    icon_elem = root.find("icon")
    inventorySlot_elem = root.find("inventorySlot")
    parsedItem = {
        "id": int(root.attrib['id']),
        "name": root.findtext("name"),
        "level": int(root.findtext("level")),
        "quality_id": int(quality_elem.attrib['id']),
        "quality_name": quality_elem.text,
        "class_id": int(class_elem.attrib['id']),
        "class_name": class_elem.text,
        "subclass_id": int(subclass_elem.attrib['id']),
        "subclass_name": subclass_elem.text,
        "icon_displayId": int(icon_elem.attrib['displayId']),
        "icon_name": icon_elem.text,
        "inventorySlot_id": int(inventorySlot_elem.attrib['id']),
        "inventorySlot_name": inventorySlot_elem.text,
        "htmlTooltip": root.findtext("htmlTooltip"),
        "link": root.findtext("link")
    }
    json_str = root.findtext("json")
    if (json_str):
        json_filepath = json_dir.joinpath("{}.json".format(parsedItem['id']))
        if(not json_filepath.exists() or overwrite_json):
            json_filepath.write_text('{' + json_str + '}')

    jsonequip_str = root.findtext("jsonEquip")
    if (jsonequip_str):         
            jsonequip_filepath = jsonequip_dir.joinpath("{}.equip.json".format(parsedItem['id']))
            if(not jsonequip_filepath.exists() or overwrite_json):
                jsonequip_filepath.write_text('{' + jsonequip_str + '}')
    return parsedItem

fail_count = 0
parsed_item_xml = []
with concurrent.futures.ThreadPoolExecutor(max_workers=(os.cpu_count() - 1)) as executor:
    futures = [executor.submit(parse_item_xml, p) for p in item_xml_paths]
    for future in tqdm(iterable=concurrent.futures.as_completed(futures), desc="Parsing Item XML", total=len(futures), leave=False):
        try:
            fres = future.result()
        except:
            fail_count += 1
        else:
            parsed_item_xml.append(fres)
items = pd.DataFrame.from_records(parsed_item_xml, index='id')

In [ ]:
def parse_jsonequip(fp):
        with fp.open() as f:
            data = json.load(f)
            data["item_id"] = int(Path(fp.stem).stem)
            return data

jsonequips = []

with concurrent.futures.ThreadPoolExecutor(max_workers=(os.cpu_count() - 1)) as executor:
    futures = []
    for fp in tqdm(iterable=jsonequip_dir.iterdir(), desc="Starting jsonequip parse futures", leave=False):
        futures.append(executor.submit(parse_jsonequip, fp))

    for future in tqdm(iterable=concurrent.futures.as_completed(futures), desc="Parsing item jsonequips", total=len(futures), leave=False):
        df = future.result()
        jsonequips.append(df)

jsonequips = pd.DataFrame.from_records(jsonequips).drop(columns={'appearances','displayid'})
jsonequips.set_index('item_id', inplace=True)
jsonequips

In [ ]:
# TODO Convert jsonequips to long format
temp3 = "abc"
print(f'{temp3=}')

In [ ]:
def get_weapon_dps(item_id):
        try:
            row = jsonequips.loc[item_id]
        except:
            return pd.NA
        else:
            if pd.notna(row['dps']):
                return row['dps']
            if pd.notna(row['mledps']):
                print("used mledps")
                return row['mledps']
            if pd.notna(row['rgddps']):
                print("used rgddps")
                return row['rgddps']
            # TODO Calculate from damage and speed numbers if all else fails
            return pd.NA

weapons = items[items['class_name'] == 'Weapons'][['subclass_name','name', 'level','quality_id','quality_name']].reset_index()
weapons.rename(columns={'subclass_name':'subclass', 'quality_name':'quality', 'level':'item level', 'id':'item_id'}, inplace=True)

weapons['quality_color'] = pd.Series([(lambda qid: item_qualities_palette[qid])(qid) for qid in weapons['quality_id']])
weapons['dps'] = pd.Series([get_weapon_dps(item_id) for item_id in weapons['item_id']])
weapons = weapons[weapons['dps'].notna()].sort_values('item_id', ascending=True).reset_index().drop(columns='index')
weapons

In [ ]:
weapons_by_subclass = {}
for subclass in weapons['subclass'].drop_duplicates().sort_values():
    weapons_by_subclass[subclass] = weapons[weapons['subclass'] == subclass]

In [ ]:
sns.set_theme(rc={'figure.figsize':(16,9)})
sns.lineplot(data=weapons_by_subclass['Daggers'], x='item level',y='dps', hue='quality_color', legend=None, sort=True).set_title('Daggers')